In [ ]:
# default_exp visualizations

In [ ]:
#export
import sys
sys.path.append('/Users/constantin/workspace/EmpiRe/MS-EmpiRe_Python/')
from ms_empire.diffquant_utils import *

In [ ]:
#export
import matplotlib.pyplot as plt
import numpy as np

def plot_pvals(result_df):
    pvals = result_df["peptide_pval"].to_list()
    plt.hist(pvals,99,cumulative=True,density=True, histtype='step')
    x = np.linspace(0,1,100)
    plt.plot(x, x)
    plt.show()

In [ ]:
#export
from scipy.stats import norm
import matplotlib.pyplot as plt

def plot_bgdist(bgdist):
    fc2counts_rescaled = tranform_fc2count_to_fc_space(bgdist.fc2counts, bgdist.cumulative[-1],1/100.0)
    
    plt.bar(list(fc2counts_rescaled.keys()), fc2counts_rescaled.values(),width=0.01,color='g')
    axes2 = plt.twinx()
    x = np.linspace(-4, 4, 1000)
    axes2.plot(x, norm.pdf(x, 0, bgdist.SD)/1.15)
    axes2.set_ylim(0.0, 0.4)
    plt.show()

def tranform_fc2count_to_fc_space(fc2counts, num_fcs, rescale_factor):
    fc2counts_fcscales = {}
    for fc, count in fc2counts.items():
        fc2counts_fcscales[fc*rescale_factor] = count/num_fcs

    return fc2counts_fcscales

In [ ]:
#export
#interactive
import matplotlib.pyplot as plt
from scipy import stats

def plot_betweencond_fcs(df_c1_normed, df_c2_normed, get_median):
    if get_median:
        df_c1_normed = df_c1_normed.median(axis = 1, skipna = True).to_frame()
        df_c2_normed = df_c2_normed.median(axis = 1, skipna = True).to_frame()
    both_idx = df_c1_normed.index.intersection(df_c2_normed.index)
    df1 = df_c1_normed.loc[both_idx]
    df2 = df_c2_normed.loc[both_idx]

    for col1 in df1.columns:
        for col2 in df2.columns:
            diff_fcs = df1[col1].to_numpy() - df2[col2].to_numpy()
            median = np.nanmedian(diff_fcs)
            #plt.axvline(mode, color = 'blue')
            plt.axvline(median, color = 'red')
            plt.hist(diff_fcs,99,density=True, histtype='step', range=(-3.5,3.5))
    plt.xlabel("log2(fc)")
    plt.show()

In [ ]:
#export
#interactive
import matplotlib.pyplot as plt
from scipy import stats
import itertools

def plot_withincond_fcs(df, xlim = None):
    combs = list(itertools.combinations(df.columns, 2))
    for cpair in combs:
        col1 = cpair[0]
        col2 = cpair[1]
        diff_fcs = df[col1].to_numpy() - df[col2].to_numpy()
        mode = stats.mode(diff_fcs, nan_policy='omit')[0][0]
        median = np.nanmedian(diff_fcs)
        #plt.axvline(mode, color = 'blue')
        #plt.axvline(median, color = 'red')
        plt.hist(diff_fcs,99,density=True, histtype='step',range=(-2,2))
        plt.xlabel("log2 peptide fcs")
        
    if xlim is not None:
        plt.xlim(xlim) 
    plt.show()

In [ ]:
#export
import matplotlib.pyplot as plt
def scatter_df_columns(merged_df, log_axes = False):
    col = (0.2, 0.4, 0.6, 0.1)
    ref_columns = list(filter(lambda x : "_ref" in x, merged_df.columns.to_list())) #filter the reference columns from the merged df

    for ref in ref_columns:
        compare = ref.replace("_ref", "")
        ax_p = merged_df.plot.scatter(x=ref,y=compare, color = col)
        corr = merged_df[ref].corr(merged_df[compare])
        plt.title(f"{ref} vs. {compare} corr {corr}")
        x = np.linspace(0,merged_df[ref].max(),100)
        plt.plot(x, x)
        if log_axes:
            plt.xscale('log')
            plt.yscale('log')
        plt.show()

In [ ]:
#export
import matplotlib.pyplot as plt
def plot_cumhist_dfcols(merged_df):
    col = (0.2, 0.4, 0.6, 0.4)
    ref_columns = list(filter(lambda x : "_ref" in x, merged_df.columns.to_list())) #filter the reference columns from the merged df

    for ref in ref_columns:
        compare = ref.replace("_ref", "")
        plt.hist(merged_df[ref], 100, density=True, histtype='step', label='reference')
        plt.hist(merged_df[compare], 100, density=True, histtype='step',label='compare')
        corr = merged_df[ref].corr(merged_df[compare])
        plt.title(f"{ref} vs. {compare} corr {corr}")
        plt.show()

In [ ]:
#export
from matplotlib_venn import venn2
from matplotlib import pyplot as plt
def compare_peptid_protein_overlaps(protein_ref, protein_comp, peptide_ref, peptide_comp):
    protIDs_ref = set(protein_ref["protein"].to_list())
    protIDs_comp = set(protein_comp["protein"].to_list())
    venn2([protIDs_ref, protIDs_comp], ('protIDs_ref', 'protIDs_comp'))
    plt.show()
    pepIDs_ref = set(peptide_ref["ion"].to_list())
    pepIDs_comp = set(peptide_comp["ion"].to_list())
    venn2([pepIDs_ref, pepIDs_comp], ('pepIDs_ref', 'pepIDs_comp'))
    plt.show()

In [ ]:
#export
import matplotlib.pyplot as plt
import seaborn as sns

def plot_fold_change(df, key1, key2):
    to_plot = df.copy()
    to_plot[f'Ratio ({key1}/{key2})'] = np.log2(to_plot[key1] / to_plot[key2])
    to_plot[f'Inten_{key1}'] = np.log10(to_plot[key1])

    species = 'Human'
    val = to_plot.loc[to_plot['species']==species, f'Ratio ({key1}/{key2})'].values
    val = val[~np.isnan(val)&~np.isinf(val)&~np.isneginf(val)]
    print(f'Species={species}, n={len(val)}, median={np.median(val)}, dev={np.std(val)}')
    species='Ecoli'
    val = to_plot.loc[to_plot['species']==species, f'Ratio ({key1}/{key2})'].values
    val = val[~np.isnan(val)&~np.isinf(val)&~np.isneginf(val)]
    print(f'species={species}, n={len(val)}, median={np.median(val)}, dev={np.std(val)}')

    plt.figure(figsize=(7,7))
    ax = sns.scatterplot(x=f'Ratio ({key1}/{key2})', y=f'Inten_{key1}', hue="species", data=to_plot, alpha=0.5)
    plt.title('Fold Change')
    plt.xlim([-4.5, 6.5])
    #plt.ylim([6,11.5])
    plt.show()


In [ ]:
#export
#interactive
import matplotlib.pyplot as plt

import numpy as np

def volcano_plot(result_df, fc_header = "log2fc", fdr_header = "fdr", significance_cutoff = 0.05, log2fc_cutoff = 0.5,ybound = None, xbound =None):
    result_df[fdr_header] = result_df[fdr_header].replace(0, np.min(result_df[fdr_header].replace(0, 1.0)))
    fdrs = result_df[fdr_header].to_numpy()
    fcs = result_df[fc_header].to_numpy()
    sighits_down = sum((fdrs<significance_cutoff) & (fcs <= -log2fc_cutoff))
    sighits_up = sum((fdrs<significance_cutoff) & (fcs >= log2fc_cutoff))
    plt.title(f"{sighits_up} up, {sighits_down} down of {len(fcs)}")
    plt.scatter(result_df[fc_header],-np.log10(result_df[fdr_header]),s=10, c='grey', alpha = 0.1)
    plt.xlabel('log2 FC',fontsize = 14)
    plt.ylabel('-log10 FDR',fontsize = 14)

    if ybound==None:
        plt.ylim(0,max(-np.log10(result_df[fdr_header]))+0.5)
    else:
        plt.ylim(ybound)
    if significance_cutoff>0:
        plt.axhline(y=-np.log10(significance_cutoff), color='g', linestyle='-')
    if log2fc_cutoff >0:
        plt.axvline(x=log2fc_cutoff, color='g', linestyle='-')
        plt.axvline(x=-log2fc_cutoff, color='g', linestyle='-')
    maxfc = max(abs(result_df[fc_header]))+0.5
    if xbound==None:
        plt.xlim(-maxfc,maxfc)
    else:
        plt.xlim(xbound)
    plt.show()


In [ ]:
#export
#interactive
import seaborn as sns
import matplotlib.pyplot as plt

def beeswarm_ion_plot(protein,diffresults_df,normed_df, sample2cond,saveloc = None,ion_header = 'ion'):
    diffresults_line = diffresults_df.loc[protein]
    fdr = diffresults_line["fdr"]
    value_vars = set.intersection(set(normed_df.columns), set(sample2cond.keys()))
    protein_df = normed_df.loc[[protein]]
    df_melted = pd.melt(protein_df, value_vars= value_vars, id_vars=[ion_header], value_name="intensity", var_name="sample")
    df_melted["condition"] = [sample2cond.get(x) for x in df_melted["sample"]]
    pal2 = [(0.94, 0.94, 0.94),(1.0, 1.0, 1.0)]
    ax = sns.boxplot(x="ion", y="intensity", hue="condition", data=df_melted, palette=pal2)
    ax = sns.swarmplot(x="ion", y="intensity", hue="condition", data=df_melted, palette="Set2", dodge=True)
    handles, labels = ax.get_legend_handles_labels()

    l = plt.legend(handles[2:4], labels[2:4])

    plt.xticks(rotation=90)
    if "gene" in diffresults_df.columns:
        gene = diffresults_line["gene"].values[0]
        plt.title(f"{gene} ({protein}) FDR: {fdr:e.1}")
    else:
        plt.title(f"{protein} FDR: {fdr:e}")
    if saveloc is not None:
        plt.savefig(saveloc)
    
    plt.show()

In [ ]:
#export
import numpy.ma as ma
import scipy.cluster.hierarchy as hierarchy

def compare_direction(array1, array2):
    identical_elements  = array1 == array2
    num_same_direction = np.sum(identical_elements)
    return num_same_direction

def compare_correlation(array1, array2):
    corr = ma.corrcoef(ma.masked_invalid(array1), ma.masked_invalid(array2))[0][1]
    return corr

def get_condensed_distance_matrix(arrays, compare_function):

    res = np.ones(int(len(arrays) * (len(arrays)-1)/2))
    count = 0
    for i in range(len(arrays)):
        for j in range(i+1, len(arrays)):
            array1 = arrays[i]
            array2 = arrays[j]
            distance = 1/compare_function(array1, array2)
            res[count] = distance
            count+=1

    return res

def clustersort_numerical_arrays(arrays, names , cluster_method ='average',compare_function = compare_direction):
    condensed_distance_matrix = get_condensed_distance_matrix(arrays, compare_function)
    linkage_matrix = hierarchy.linkage(condensed_distance_matrix, method = cluster_method)
    sorted_array_idxs = hierarchy.leaves_list(linkage_matrix)

    sorted_array = [arrays[x] for x in sorted_array_idxs]
    sorted_names = [names[x] for x in sorted_array_idxs]
    return sorted_array, sorted_names, linkage_matrix








In [ ]:
#export
import numpy as np
def compare_direction(array1, array2):
    identical_elements  = array1 == array2
    num_same_direction = np.sum(identical_elements)
    return num_same_direction


In [ ]:
#export
import numpy.ma as ma
def compare_correlation(array1, array2):
    corr = ma.corrcoef(ma.masked_invalid(array1), ma.masked_invalid(array2))[0][1]
    return corr

In [ ]:
#export

import scipy.cluster.hierarchy as hierarchy
import scipy.spatial.distance as dist
def clustersort_numerical_arrays(arrays, names , cluster_method ='average',compare_function = compare_direction):
    #condensed_distance_matrix = get_condensed_distance_matrix(arrays, compare_function)
    condensed_distance_matrix = dist.pdist(arrays, lambda u, v: 1/(compare_function(u,v)+1))
    linkage_matrix = hierarchy.linkage(condensed_distance_matrix, method = cluster_method)
    sorted_array_idxs = hierarchy.leaves_list(linkage_matrix)

    sorted_array = np.array([arrays[x] for x in sorted_array_idxs])
    sorted_names = np.array([names[x] for x in sorted_array_idxs])
    return sorted_array, sorted_names, linkage_matrix


In [ ]:
#export

import pandas as pd
import numpy as np
import os
def get_clustered_dataframe(overview_df, cluster_method ='average',compare_function = compare_direction, clust_rows = True, clust_columns = True):

    df_numbered = overview_df.select_dtypes(include=np.number)
    contains_floats = ['float' in str(x) for x in df_numbered.dtypes]
    type = 'float' if True in contains_floats else 'int' 
    df_numbered = df_numbered.astype(type) #ensure that the df has no mixed types

    rows = df_numbered.to_numpy()
    rownames = list(df_numbered.index)
    colnames = list(df_numbered.columns)

    if clust_rows:
        if(len(rownames)>10000):
            print(f"large number of rows, skipping cluster step of rows to avoid long runtime.")
        else:
            print(f"clustering on {len(rownames)} rows")
            rows, rownames, _ = clustersort_numerical_arrays(rows, rownames, cluster_method, compare_function)
    if clust_columns:
        if(len(colnames)>10000):
            print(f"large number of columns, skipping cluster step of columns to avoid long runtime")
        else:
            print(f"clustering on {len(colnames)} columns")
            columns, colnames,_ = clustersort_numerical_arrays(rows.T, colnames, cluster_method, compare_function)
            rows = columns.T
    print("finished clustering")
    df_clustered = pd.DataFrame(rows, index= rownames, columns= colnames ).reset_index()
    return df_clustered


In [ ]:
#export

import holoviews as hv
import pandas as pd
hv.extension('bokeh')

def get_heatmap(overview_df, diffresults_folder = os.path.join(".", "diffresults")):
    clustered_df = get_clustered_dataframe(overview_df)
    if diffresults_folder != None:
        clustered_df.to_csv(os.path.join(diffresults_folder, "regulation_overview.tsv"), sep = "\t", index = None)
    plot_df = pd.melt(clustered_df, id_vars='index')
    heatmap = hv.HeatMap(plot_df, label='Regulation overview')
    return heatmap
    


In [ ]:
#export
import re
import os
def get_sample_overview_dataframe(diffresults_folder = os.path.join(".", "diffresults"), condpairs_to_compare = []):
    """
    goes through the results folder and extracts up- and downregulated genes for each (specified) condition comparison
    """

    if len(condpairs_to_compare) == 0:
        condpairs_to_compare = [f.replace(".results.tsv", "").split("_VS_") for f in os.listdir(diffresults_folder) if re.match(r'.*results.tsv', f)]

    dfs = []
    count = 0
    for row in condpairs_to_compare:
        c1 = row[0]
        c2 = row[1]
        results = initialize_result_dataframe(c1, c2, diffresults_folder)
        if(type(results) == type(None)):
            continue
        site_df = results
        positive_sites = list(set(site_df[(site_df["fdr"]<0.05) & (site_df["log2fc"]>0.5)]["site_id"]))
        negative_sites = list(set(site_df[(site_df["fdr"]<0.05) & (site_df["log2fc"]<-0.5)]["site_id"]))
        df_loc = pd.DataFrame([[1 for x in range(len(positive_sites))]+[-1 for x in range(len(negative_sites))]],columns=positive_sites+negative_sites)
        df_loc["condpair"] = get_condpairname([c1, c2])
        #df_loc["num_regulated"] = len(positive_sites) + len(negative_sites)
        dfs.append(df_loc)
        #print(count)
        count+=1

    result_df = pd.concat(dfs)
    result_df = result_df.replace(np.nan, 0).set_index("condpair")

    return result_df


In [ ]:
#export
import pandas as pd
import os
import numpy as np

def initialize_result_dataframe(cond1, cond2, diffresults_folder = os.path.join(".", "diffresults")):
    """
    reads the results dataframe for a given condpair
    """
    condpair = get_condpairname([cond1, cond2])
    diffresults = os.path.join(diffresults_folder, f"{condpair}.results.tsv")

    try:
        diffprots = pd.read_csv(diffresults, sep = "\t")
    except:
        print(f"no quantfiles found for {condpair}!")
        return None
    diffprots = diffprots[(diffprots["condpair"] == condpair)]

    diffprots["-log10fdr"] = -np.log10(diffprots["fdr"])
    diffprots = diffprots.set_index("protein")
    
    return diffprots

In [ ]:
#export
import pandas as pd
import os
import numpy as np

def initialize_normed_peptides(cond1, cond2, diffresults_folder = os.path.join(".", "diffresults")):
    """
    reads normed peptide data for a given condpair
    """
    condpair = get_condpairname([cond1, cond2])
    normed_peptides_tsv = os.path.join(diffresults_folder, f"{condpair}.normed.tsv")
    try:
        normed_peptides = pd.read_csv(normed_peptides_tsv, sep = "\t")
    except:
        print(f"no normed peptides found for {condpair}!")
        return None
    
    numeric_cols = list(normed_peptides.select_dtypes(include=np.number).columns)
    #available_vals = list(set(samplemap_df["sample"].values).intersection(set(normed_peptides.columns)))
    normed_peptides[numeric_cols] = np.log2(normed_peptides[numeric_cols].replace(0, np.nan))
    normed_peptides = normed_peptides.set_index("protein")
    return normed_peptides


In [ ]:
#export
import pandas as pd

def initialize_sample2cond(samplemap):
    samplemap_df = pd.read_csv(samplemap, sep = "\t")
    sample2cond = dict(zip(samplemap_df["sample"], samplemap_df["condition"]))
    return samplemap_df, sample2cond